# Imports and loads

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [0]:
!kaggle competitions download -c bluebook-for-bulldozers -p /content
!unzip *.zip
!unzip Train.zip
!ls *.zip *.csv

In [0]:
%time df = pd.read_csv('Train.csv', low_memory=False, parse_dates=["saledate"])

CPU times: user 1min 11s, sys: 622 ms, total: 1min 12s
Wall time: 1min 13s


# Lesson 1

### Data exploration

In [0]:
# 'summary'
df.describe()

In [0]:
df.info()

In [0]:
df.shape()

In [0]:
# columns
df.columns

In [0]:
# quantiles
df.quantile([0,0.25,0.5,0.75,1])

In [0]:
# head
#df.head()

# tail
df.tail().T # == df.tail().transpose()

In [0]:
fld = df.saledate
fld.dt.year

### Training a base model

In [0]:
# função que altera os parametros de display do pandas
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(df.isnull().sum().sort_index()/len(df))

#### Pre-processing data

In [0]:
# função que cria uma coluna "_na" e preenche os dados com a mediana
def fix_missing(df, col, name):
  if is_numeric_dtype(col):
    if pd.isnull(col).sum(): df[name+"_na"] = pd.isnull(col)
    df[name] = col.fillna(col.median())

In [0]:
def numericalize(df, col, name, max_n_cat):
  if not is_numeric_dtype(col) and (max_n_cat is None or col.nunique()>max_n_cat):
    df[name] = col.cat.codes+1

After running `add_datepart`, it added many numerical columns and removed saledate column.      

This is not quite enough to get passed the error we saw earlier as we still have other columns that contain string values. Pandas has a concept of a category data type, but by default it would not turn anything into a category for you. Fast.ai provides a function called `train_cats` which creates categorical variables for everything that is a String. Behind the scenes, it creates a column that is an integer and it is going to store a mapping from the integers to the strings. `train_cats` is called “train” because it is training data specific.           

It is important that validation and test sets will use the same category mappings (in other words, if you used 1 for “high” for a training dataset, then 1 should also be for “high” in validation and test datasets). For validation and test dataset, use `apply_cats` instead.

In [0]:
df['SalePrice'] = np.log(df['SalePrice'])

# Função do fastai que formata datas
add_datepart(df,'saledate')

# Função que converte strings -> numérico
train_cats(df)

#df1, y, nas = proc_df(df, 'SalePrice')

In [0]:
# função 'levels' das variáveis categóricas (exploratório)
#df1.UsageBand.cat.categories

# método para alterar os níveis das vars categóricas
#df1.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

#Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. 
#Optionally, we can replace the text categories with numbers, which will make this variable non-categorical
#df1.UsageBand = df1.UsageBand.cat.codes

In [0]:
# R equivalent: is.null
#df1.isnull()

# proporção de nulos por coluna
#df1.isnull().sum().sort_index()/len(df)

#### Splitting train, test      
avoid overfitting

In [0]:
# função que retorna dois datasets cortados na posição n (não aleatório)
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [0]:
n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df1)-n_valid
raw_train, raw_valid = split_vals(df1, n_trn)
X_train, X_valid = split_vals(df1, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    rmses = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid)]
    scores = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): scores.append(m.oob_score_)
    print("RMSEs:")
    print(rmses)
    print("R-squared:")
    print(scores)

In [0]:
model = RandomForestRegressor(n_jobs=-1)
%time model.fit(X_train, y_train)

In [0]:
# RMSE e R^2
print_score(model)

RMSEs:
[0.09071769696554534, 0.24845082174679786]
R-squared:
[0.9828003797297382, 0.889762628594108]


### Speeding things up       
using subsets for faster learning

In [0]:
# 'subset' of proc_df extracts a random sample from df. Split_vals gets a non-random sample (1:n, and n:)
df_trn, y_trn, nas = proc_df(df, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [0]:
model = RandomForestRegressor(n_jobs=-1)
%time model.fit(X_train, y_train)

In [0]:
# RMSE e R^2
print_score(model)

RMSEs:
[0.11211366828424382, 0.38517428196090564]
R-squared:
[0.9724867244284848, 0.7350508459212594]


#### Estimating a Single Tree

In [0]:
# 'n_estimators' = numero de arvores, 'max_depth' = numero de cortes (splits)
# 'bootstrap' = Whether bootstrap samples are used when building trees. If False, the whole datset is used to build each tree.
model = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
%time model.fit(X_train, y_train)

CPU times: user 153 ms, sys: 14 ms, total: 167 ms
Wall time: 194 ms


RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [0]:
print_score(model)

RMSEs:
[0.5241183153725063, 0.5798358906438746]
R-squared:
[0.39871018762448407, 0.39957583669327423]


In [0]:
# Representação gráfica de uma árvore
draw_tree(model.estimators_[0],df_trn, precision=3)

#### Creating a bigger tree

In [0]:
# max_depth='None'
model = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
%time model.fit(X_train, y_train)
print_score(model)
# Worse than our original model

### Bagging

In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [0]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds

array([[9.25913, 9.71112, 9.04782, ..., 8.92266, 9.07681, 9.07681],
       [9.30565, 9.3501 , 9.07681, ..., 9.39266, 9.43348, 9.43348],
       [8.92266, 9.3501 , 9.15905, ..., 9.51044, 9.51044, 9.51044],
       ...,
       [8.92266, 9.15905, 9.15905, ..., 8.92266, 8.92266, 8.92266],
       [9.97581, 9.15905, 9.04782, ..., 9.25913, 9.74097, 9.74097],
       [8.9872 , 9.15905, 8.9872 , ..., 9.9988 , 9.6486 , 9.6486 ]])

In [0]:
preds[:,0], np.mean(preds[:,0]), y_valid[0]

(array([9.25913, 9.30565, 8.92266, 9.04782, 8.92266, 8.92266, 8.92266, 8.92266, 9.97581, 8.9872 ]),
 9.118889906280428,
 9.104979856318357)

In [0]:
_ = matplotlib.pyplot.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(10)])

In [0]:
m1 = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m1.fit(X_train, y_train)

m2 = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m2.fit(X_train, y_train)

m3 = RandomForestRegressor(n_estimators=60, n_jobs=-1)
m3.fit(X_train, y_train)

print_score(m1), print_score(m2), print_score(m3)

#### Out-of-bag (OOB) score        
Is our validation set worse than our training set because we're over-fitting, or because the validation set is for a different time period, or a bit of both? With the existing information we've shown, we can't tell. However, random forests have a very clever trick called out-of-bag (OOB) error which can handle this (and more!)             
The idea is to calculate error on the training set, but only include the trees in the calculation of a row's error where that row was not included in training that tree. This allows us to see whether the model is over-fitting, without needing a separate validation set.             
This also has the benefit of allowing us to see whether our model generalizes, even if we only have a small amount of data so want to avoid separating some out to create a validation set.

In [0]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

RMSEs:
[0.09670541378683264, 0.3632253231472383]
R-squared:
[0.9795295802570879, 0.7643864778020514, 0.8534221298338784]


### Reducing Overfitting

In [0]:
df_trn, y_trn, nas = proc_df(df, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

#### Subsampling

In [0]:
set_rf_samples(20000)
# undo line above: reset_rf_samples()

In [0]:
m = RandomForestRegressor(n_jobs=-1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)
# fast performance with lower error

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 11.2 s, sys: 124 ms, total: 11.3 s
Wall time: 7.28 s
RMSEs:
[0.2404522224840547, 0.2773636898085969]
R-squared:
[0.8791650915830417, 0.8626125109303167, 0.8666484502884869]


In [0]:
m1 = RandomForestRegressor(n_estimators=20, n_jobs=-1, oob_score=True)
m1.fit(X_train, y_train)

m2 = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m2.fit(X_train, y_train)

m3 = RandomForestRegressor(n_estimators=60, n_jobs=-1, oob_score=True)
m3.fit(X_train, y_train)

print_score(m1), print_score(m2), print_score(m3)

RMSEs:
[0.23201202227219894, 0.2682992529145708]
R-squared:
[0.8874991496547435, 0.8714456133190257, 0.875566790341897]
RMSEs:
[0.2268143235144569, 0.26104130070702325]
R-squared:
[0.8924833360884015, 0.8783067685527927, 0.880969378290751]
RMSEs:
[0.22552969301870618, 0.2617220663850295]
R-squared:
[0.8936977924203511, 0.8776712170170742, 0.8822956883696145]


(None, None, None)

#### Tree building parameters

In [0]:
reset_rf_samples()

In [0]:
# função que detecta profundidade da árvore
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

In [0]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

RMSEs:
[0.0784978237895985, 0.2365577321207158]
R-squared:
[0.9871219513963191, 0.9000639292753713, 0.9084567605662913]


In [0]:
t=m.estimators_[0].tree_
dectree_max_depth(t)

46

In [0]:
m2 = RandomForestRegressor(n_estimators=40, min_samples_leaf=5, n_jobs=-1, oob_score=True)
m2.fit(X_train, y_train)
print_score(m2)

RMSEs:
[0.14067346329265226, 0.23425906355398388]
R-squared:
[0.958642032710944, 0.9019966819294172, 0.9070834771469374]


In [0]:
t=m2.estimators_[0].tree_
dectree_max_depth(t)

38

In [0]:
m3 = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m3.fit(X_train, y_train)
print_score(m3)

### OOB Errors with sklearn example

The example below demonstrates how the OOB error can be measured at the addition of each new tree during training. The resulting plot allows a practitioner to approximate a suitable value of n_estimators at which the error stabilizes.

Source: T. Hastie, R. Tibshirani and J. Friedman, “Elements of Statistical Learning Ed. 2”, p592-593, Springer, 2009.

In [0]:
from collections import OrderedDict

In [0]:
r_state = 333

ensemble_regs = [("RandomForestRegressor, max_features='sqrt'", 
                  RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='sqrt', 
                                        warm_start=True, oob_score=True, random_state=r_state)),
                 ("RandomForestRegressor, max_features='log2'",
                  RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features='log2', 
                                        warm_start=True, oob_score=True, random_state=r_state)),
                 ("RandomForestRegressor, max_features=None",
                  RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features=None, 
                                        warm_start=True, oob_score=True, random_state=r_state))]

In [0]:
# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_regs)

In [0]:
# Range of `n_estimators` values to explore.
min_estimators = 30
max_estimators = 200

In [0]:
for label, reg in ensemble_regs:
    for i in range(min_estimators, max_estimators + 1):
        reg.set_params(n_estimators=i)
        reg.fit(X_train, y_train)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - reg.oob_score_
        error_rate[label].append((i, oob_error))

In [0]:
# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, reg_err in error_rate.items():
    xs, ys = zip(*reg_err)
    plt.plot(xs, ys, label=label)

In [0]:
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

# Lesson 2

## Setup

In [0]:
set_plot_sizes(12,14,16)

def split_vals(a,n): return a[:n], a[n:]
n_valid = 12000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

## Confidence based on tree variance